In [11]:
# LCC implementation
import numpy as np
import galois

In [74]:
# utils
def modulo_inverse(x, p):
    return pow(x, -1, p)

def finite_field_division(quotient, dividend, p):
    # find modulo inverse of dividend and multiply with the quotient
    mod_inv_div = modulo_inverse(dividend, p)
    return (mod_inv_div * quotient) % p

def poly_coefficients(evaluated_points, evaluation_points, p):
    vander_matrix = np.vander(evaluation_points, increasing=True) % p
    galois_field = galois.GF(p)
    vander_matrix = galois_field(vander_matrix)
    inv_vander_matrix = np.linalg.inv(vander_matrix)
    
    evaluated_points_in_field = galois_field(np.asarray(evaluated_points)[:, np.newaxis])
    
    return inv_vander_matrix @ evaluated_points_in_field

In [75]:
# lagrange coded computing polynomial 
class LCCPoly(object):
    def __init__(self, beta_arr, secret_arr, K, T, p):
        self.K = K
        self.T = T
        self.p = p
        self.beta_arr = beta_arr
        self.secret_arr = secret_arr
        self.degree = K + T
        
        self._random_arr = []
        self.__fill_random_arr()
    
    def __fill_random_arr(self):
        for j in range(self.K, self.K + self.T):
            self._random_arr.append(np.random.randint(0, high=self.p))
    
    @property
    def random_arr(self):
        return self._random_arr
        
    def __call__(self, evaluation_point, *args, **kwargs):
        result = 0
        for j in range(self.K):
            curr_secret = self.secret_arr[j]
            curr_mul = 1
            for k in range(self.K + self.T):
                if k == j:
                    continue
                else:
                    first = (evaluation_point - self.beta_arr[k]) % self.p
                    second = (self.beta_arr[j] - self.beta_arr[k]) % self.p
                    curr_mul = (curr_mul * finite_field_division(first, second, self.p)) % self.p
            result = (result + ((curr_secret * curr_mul) % self.p)) % self.p
        for j in range(self.K, self.K + self.T):
            curr_random = self._random_arr[j - self.K]
            curr_mul = 1
            for k in range(self.K + self.T):
                if k == j:
                    continue
                else:
                    first = (evaluation_point - self.beta_arr[k]) % self.p
                    second = (self.beta_arr[j] - self.beta_arr[k]) % self.p
                    curr_mul = (curr_mul * finite_field_division(first, second, self.p)) % self.p
            result = (result + ((curr_random * curr_mul) % self.p)) % self.p
        return result
    
class InterpolatedPoly(object):
    def __init__(self, evaluated_points, evaluation_points, p):
        self.evaluated_points = evaluated_points
        self.evaluation_points = evaluation_points
        self.p = p
        
    def __call__(self, evaluation_point, *args, **kwargs):
        result = 0
        for j in range(len(self.evaluated_points)):
            curr_secret = self.evaluated_points[j]
            curr_mul = 1
            for k in range(len(self.evaluated_points)):
                if k == j:
                    continue
                else:
                    first = (evaluation_point - self.evaluation_points[k]) % self.p
                    second = (self.evaluation_points[j] - self.evaluation_points[k]) % self.p
                    curr_mul = (curr_mul * finite_field_division(first, second, self.p)) % self.p
            result = (result + ((curr_secret * curr_mul) % self.p)) % self.p
        return result

In [78]:
tmp_poly = LCCPoly([0, 1, 2], [3, 4], 2, 1, 7)
poly_coefficients([tmp_poly(4), tmp_poly(5), tmp_poly(2)], [4, 5, 2], 7)

GF([[3],
    [1],
    [0]], order=7)

In [80]:
poly_coefficients([tmp_poly(3), tmp_poly(6), tmp_poly(5)], [3, 6, 5], 7)

GF([[3],
    [1],
    [0]], order=7)

In [ ]:
# lagrange coded computing equation

# constants
parallelization_param_K = 2
privacy_param_T = 1
finite_field_size_p = 5




In [69]:
import galois

field = galois.GF(7)
tmp_matrix = np.asarray([[3, 4, 2],
                         [6, 5, 1],
                         [0, 3, 4]])
tmp_matrix = field(tmp_matrix)
inv_tmp_matrix = np.linalg.inv(tmp_matrix)

/home/umityigitbsrn/miniconda3/envs/pytorch-stable/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [70]:
tmp_matrix

GF([[3, 4, 2],
    [6, 5, 1],
    [0, 3, 4]], order=7)

In [72]:
inv_tmp_matrix @ tmp_matrix

GF([[1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]], order=7)